# Analysis of Critical Role's Campaign 2: Mighty Nein.

This notebook takes care of the data manipulation for Critical Role's Campaign 2 (also refered to as Mighty Nein data).

**Data sources**

- [CRStats](https://www.critrolestats.com/).

## Import modules
Libraries used throughout the project.

In [1]:
import pandas as pd
import numpy as np

## Loading the data
There are several datasets that needs to be open and clean. They are dealt with in separate subsections. Note that a preliminary data manipulation has been done in Excel, prior to the one executed in this notebook, to have the data into different `csv` files.

### Data on all rolls

The first dataset loaded is `AllRollsMN.csv`, where data about all rolls during the campaign are stored. The columns are:
- `Episode`, the episode number;
- `Time`, the timestamp of the roll;
- `Character`, the character who rolled;
- `Type of Roll`, what kind of roll is;
- `Total Value`, the total number of the roll (with added bonuses and modifiers);
- `Natural Value`, the natural value of the dice roll;
- `Crit?`, is it a critical roll?
- `Damage Dealt`, the damage dealt by attack;
- `# Kills`, how many enemy killed with that roll;
- `Notes`, comments on the roll.

In [2]:
rolls = pd.read_csv('MightyNein/AllRollsMN.csv', sep=';')

In [3]:
rolls.columns

Index(['Episode', 'Time', 'Character', 'Type of Roll', 'Total Value',
       'Natural Value', 'Crit?', 'Damage Dealt', '# Kills', 'Notes'],
      dtype='object')

In [4]:
rolls.dtypes

Episode           object
Time              object
Character         object
Type of Roll      object
Total Value       object
Natural Value     object
Crit?             object
Damage Dealt      object
# Kills          float64
Notes             object
dtype: object

First, we drop columns that are not useful: `Notes`, `Damage Dealt` and `Crit?`. These columns are too messy or are missing data. The `Crit?` can be obtained from an analysis of `Natural Value`, it will mean that some critical rolls on a 19 will be missed (for example a specific subclass of barbarian can crit on a 19). Moreover, the damage is analysed in a separate file.

In [5]:
rolls.drop(columns=['Damage Dealt', 'Notes', 'Crit?'], inplace=True)

In [6]:
#Let's rename some columns for simplicity
rolls.rename(columns={'Type of Roll': 'roll_type', 'Total Value': 'tot_value',
                      'Natural Value': 'nat_value', '# Kills': 'num_kills'}, inplace=True)

The `Episode` columns stores the episode number, let's turn it into a numerical variable.

In [7]:
rolls['Episode'] = rolls['Episode'].str.split(pat='E').str[1]
rolls['Episode'] = rolls['Episode'].astype('int')

In [8]:
#pd.to_datetime(rolls['Time'],format='%H:%M:%S').dt.time

Let's check that all rolls have a `Character` and a `roll_type` assigned.

In [8]:
rolls.Character.isna().sum()

0

Let's group together Nott and Veth since they are effectively the same character.

In [11]:
rolls['Character'].replace(['Nott', 'Veth'],'Nott/Veth',inplace=True)

In [13]:
rolls.roll_type.isna().sum()

0

In the column `tot_value` and  `nat_value` some rolls are stored as `Nat#`, let's remove that `Nat`, and store the rolls as floats. If a rolls is  `Unknown`, it will just be a `NaN`.

In [14]:
rolls['tot_value'] = np.where(rolls.tot_value.str[0:3] == 'Nat',
                              rolls.tot_value.str[3:],rolls.tot_value)
rolls['tot_value'].replace('Unknown',np.NaN,inplace=True)
rolls['tot_value'] = rolls['tot_value'].astype('float')
rolls['nat_value'].replace('Unknown',np.NaN,inplace=True)
rolls['nat_value'] = rolls['nat_value'].astype('float')

A weird thing is happening with some rolls, they have 'natural' values 0 or negative, which is not possible. I isolate these rolls: there are only 4 entries that display such behaviour. These will be turned into `NaN`.

In [15]:
rolls[rolls.nat_value <= 0]

,Episode,Time,Character,roll_type,tot_value,nat_value,num_kills
3440,29,00:56:46,Keg,Initiative,3.0,0.0,NaN
10916,95,03:33:31,Beau,Investigation,10.0,-2.0,NaN
11665,104,02:33:15,Caleb,Wisdom Save,7.0,-1.0,NaN
12254,113,01:57:49,Caleb,Wisdom Save,5.0,-4.0,NaN


In [16]:
rolls['nat_value'] = np.where(rolls.nat_value <= 0, np.nan,rolls.nat_value)

Most of the rolls do not result in killing an enemy, these events are seved as `NaN` in `num_kills`. To allow a smoother analysis, they are transformed to 0.

In [17]:
rolls['num_kills'].fillna(0,inplace=True)

Lastly, the `roll_type` category is quite messy. Let's clean it.

In [18]:
list_rolls_type = rolls['roll_type'].unique()
list_rolls_type.sort()
print(list_rolls_type)

['Acrobatics' 'Alchemist Kit' 'Alchemist Tools' 'Alchemy Kit'
 'Animal Handling' 'Arcana' 'Athletics' 'Attack' 'Basic Check' 'Blink'
 'Charisma' 'Charisma Save' 'Check' 'Consitution Save' 'Constitution'
 'Constitution Save' 'Counterspell' 'D20' 'Damage' 'Death Save'
 'Deception' 'Deflect Missiles' 'Dexterity' 'Dexterity Save'
 'Disguise Kit' 'Divine Intervention' 'Encounter' 'Flat Roll' 'Forgery'
 'Forgery Kit' 'Forgery Tools' 'Fragment' 'Guidance' 'Healing' 'History'
 'Hit Dice' 'Hit Points' 'Inevstigation' 'Initiative' 'Insight'
 'Intelligence' 'Intelligence Save' 'Intimidation' 'Investigation'
 'Medicine' 'Mirror Image' 'Nature' 'Other' 'Percentage' 'Perception'
 'Performance' 'Persuasion' 'Religion' 'Sleight of Hand' 'Spell Attack'
 'Stealth' 'Steath' 'Strength' 'Strength Save' 'Survival' "Thieves' Tools"
 "Tinker's Tools" 'Unknown' 'Wisdom' 'Wisdom Save' 'd100' 'd20']


In [19]:
rolls['roll_type'].replace(['Alchemist Kit', 'Alchemy Kit'], 'Alchemist Tools', inplace=True)
rolls['roll_type'].replace(['Check'], 'Basic Check', inplace=True)
rolls['roll_type'].replace(['Consitution Save'], 'Constitution Save', inplace=True)
rolls['roll_type'].replace(['Forgery Kit', 'Forgery'], 'Forgery Tools', inplace=True)
rolls['roll_type'].replace(['Inevstigation'], 'Investigation', inplace=True)
rolls['roll_type'].replace(['Steath'], 'Stealth', inplace=True)
rolls['roll_type'].replace(["Thieves' Tools"], 'Thieves Tools', inplace=True)
rolls['roll_type'].replace(["Tinker's Tools"], 'Tinker Tools', inplace=True)
rolls['roll_type'].replace(['D20','Deflect Missiles','Encounter', 'Flat Roll',
                           'Fragment', 'Mirror Image', 'Percentage','d100',
                           'd20', 'Hit Dice', 'Hit Points', 'Alchemist Tools',
                           'Disguise Kit', 'Forgery Tools', 'Thieves Tools', 'Tinker Tools',
                           'Basic Check', 'Blink'], 'Other', inplace=True)

In [20]:
list_rolls_type = rolls['roll_type'].unique()
list_rolls_type.sort()
print(list_rolls_type)

['Acrobatics' 'Animal Handling' 'Arcana' 'Athletics' 'Attack' 'Charisma'
 'Charisma Save' 'Constitution' 'Constitution Save' 'Counterspell'
 'Damage' 'Death Save' 'Deception' 'Dexterity' 'Dexterity Save'
 'Divine Intervention' 'Guidance' 'Healing' 'History' 'Initiative'
 'Insight' 'Intelligence' 'Intelligence Save' 'Intimidation'
 'Investigation' 'Medicine' 'Nature' 'Other' 'Perception' 'Performance'
 'Persuasion' 'Religion' 'Sleight of Hand' 'Spell Attack' 'Stealth'
 'Strength' 'Strength Save' 'Survival' 'Unknown' 'Wisdom' 'Wisdom Save']


Let's create a new column, to simplify the roll type further. It will differentiate into macro-types, for example checks, saves etc.

In [21]:
list_check = ['Acrobatics', 'Animal Handling', 'Arcana', 'Athletics', 'Charisma',
 'Constitution', 'Deception', 'Dexterity', 'History', 'Insight', 'Intelligence', 
 'Intimidation', 'Investigation', 'Medicine', 'Nature', 'Perception', 'Performance',
 'Persuasion', 'Religion', 'Sleight of Hand', 'Stealth', 'Strength', 'Survival', 'Wisdom']
list_save = ['Charisma Save', 'Constitution Save', 'Dexterity Save',
 'Intelligence Save', 'Strength Save', 'Wisdom Save']
list_attack = ['Attack', 'Spell Attack']
rolls['roll_macrotype']=np.where(rolls.roll_type.isin(list_check),'Check',
                                np.where(rolls.roll_type.isin(list_save),'Save',
                                        np.where(rolls.roll_type.isin(list_attack),'Attack','Other')
                                        )
                                )

Let's save the cleaned dataset.

In [22]:
rolls.to_csv('MightyNein/AllRollsMNClean.csv', index=False)